Changes to v4:
- store model for inference

Potential next steps:
- dots sample frequency
- a lot of images are ambiguous, trainings data needs to be cleaned: fit lower level function and check if residual is low -> remap to lower function
- which dot type is the best?
- deployment

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import math

NUM_IMAGES = 1000

# Create data directory if it doesn't exist

x_max = 100
x = np.linspace(-x_max, x_max, 100)


def get_random_linear(x):
    a = math.tan(random.uniform(math.radians(-89.0), math.radians(89.0)))
    b = random.uniform(-x_max, x_max)
    x_shift = random.uniform(-x_max, x_max)

    print("a:", a, "b:", b)

    return a * (x + x_shift) + b


def get_random_quadratic(x):
    log_a_min = np.log(0.001)  # minimum curvature
    log_a_max = np.log(10.0)  # maximum curvature
    log_a = random.uniform(log_a_min, log_a_max)
    a = np.exp(log_a) * random.choice([-1, 1])  # random sign
    b = math.tan(random.uniform(math.radians(-89.0), math.radians(89.0)))
    c = random.uniform(-x_max, x_max)
    x_shift = random.uniform(-x_max, x_max)

    x_shifted = x + x_shift

    return a * x_shifted**2 + b * x_shifted + c


def get_random_cubic(x):
    log_a_min = np.log(0.000001)  # minimum curvature derivative
    log_a_max = np.log(0.1)  # maximum curvature derivative
    log_a = random.uniform(log_a_min, log_a_max)
    a = np.exp(log_a) * random.choice([-1, 1])  # random sign
    b = random.uniform(0.001, 0.1) * random.choice([-1, 1])
    c = math.tan(random.uniform(math.radians(-89.0), math.radians(89.0)))
    d = random.uniform(-x_max, x_max)
    x_shift = random.uniform(-x_max, x_max)

    x_shifted = x + x_shift

    return a * x_shifted**3 + b * x_shifted**2 + c * x_shifted + d


def save_plot(x, y, i, label):
    # Create figure with square aspect ratio
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.plot(x, y, "x", linewidth=4)
    ax.set_xlim([-x_max, x_max])
    ax.set_ylim([-x_max, x_max])

    ax.set_xticks([])
    ax.set_yticks([])

    folder_name = f"data/{label}"

    os.makedirs(folder_name, exist_ok=True)

    file_name = f"{folder_name}/{i}.png"
    plt.savefig(file_name, dpi=50)

    print("label:", label)
    plt.close()  # Close the figure to prevent it from being displayed
    print(f"Plot saved to {file_name}")


for i in range(NUM_IMAGES):
    y_linear = get_random_linear(x)
    save_plot(x, y_linear, i, "linear")

    y_quadratic = get_random_quadratic(x)
    save_plot(x, y_quadratic, i, "quadratic")

    y_cubic = get_random_cubic(x)
    save_plot(x, y_cubic, i, "cubic")

Load data into fast ai `DataBlock`

In [ ]:
from fastai.vision.all import (
    DataBlock,
    ImageBlock,
    CategoryBlock,
    get_image_files,
    RandomSplitter,
    parent_label,
)
from pathlib import Path


path = Path("data")

dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
).dataloaders(path, bs=32)

dls.show_batch(max_n=20)

Train the model

In [ ]:
from fastai.vision.all import (
    vision_learner,
    resnet18,
    error_rate,
)


learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(10)

Show confusion matrix

In [ ]:
from fastai.vision.all import ClassificationInterpretation

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12, 12), dpi=60)

In [ ]:
learn.save("model_v4")

In [ ]:
# Load the saved model and verify performance

# Get performance of original learner for comparison
print("Original learner performance:")
original_results = learn.validate()
print(f"Original validation loss: {original_results[0]:.4f}")
print(f"Original error rate: {original_results[1]:.4f}")

# Create a new learner with same architecture and load the saved weights (safer than pickle)
loaded_learn = vision_learner(dls, resnet18, metrics=error_rate)
loaded_learn.load("model_v4")

# Evaluate performance of loaded model
print("\nLoaded model performance:")
loaded_results = loaded_learn.validate()
print(f"Loaded validation loss: {loaded_results[0]:.4f}")
print(f"Loaded error rate: {loaded_results[1]:.4f}")

# Check if performances match
loss_diff = abs(original_results[0] - loaded_results[0])
error_diff = abs(original_results[1] - loaded_results[1])

print("\nPerformance comparison:")
print(f"Loss difference: {loss_diff:.6f}")
print(f"Error rate difference: {error_diff:.6f}")

if loss_diff < 1e-6 and error_diff < 1e-6:
    print("✅ Model saved and loaded successfully - performances match!")
else:
    print("⚠️ Performance mismatch detected")